In [1]:
import pyscf
import pyscf.tools
import sys
sys.path.append('../../OrbitalPartitioning/orbitalpartitioning')
import orbitalpartitioning
from orbitalpartitioning import *

In [3]:
molecule = '''
He       0.0000000000      0.0000000000      0.0000000000                 
He       0.0000000000      2.0000000000      0.0000000000                 
He       2.0000000000      0.0000000000      0.0000000000                 
He       2.0000000000      2.0000000000      0.0000000000                 
He       0.0000000000      0.0000000000      5.0000000000                 
He       0.0000000000      2.0000000000      5.0000000000                 
He       2.0000000000      0.0000000000      5.0000000000                 
He       2.0000000000      2.0000000000      5.0000000000 
'''
basis = "6-31g*"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        #symmetry=   True,
        spin    =   0, # number of unpaired electrons
        charge  =   0,
        basis   =   basis)


pymol.build()
#print("symmetry: ",pymol.topgroup)
mf = pyscf.scf.RHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
#mf.conv_tol_grad = 1e-5
#mf.init_guess = "sad"

mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()
# Get data
F = mf.get_fock()
C = mf.mo_coeff
S = mf.get_ovlp()
print("size of MO coeff", C.shape)
ndocc = pymol.nelec[1]
nsing = pymol.nelec[0] - ndocc
nvirt = pymol.mol.nao - ndocc - nsing
print(ndocc)
print(nsing)
print(nvirt)
# Just use alpha orbitals
Cdocc = mf.mo_coeff[:,0:ndocc]
Csing = mf.mo_coeff[:,ndocc:ndocc+nsing]
Cvirt = mf.mo_coeff[:,ndocc+nsing:ndocc+nsing+nvirt]
np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/C_scf", C)

nbas = Cdocc.shape[0]



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = None
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/td/qpnnxwv93pq0t7bbdkh5rvzr0000gn/T/tmpdmh54urt
max_memory 4000 MB (current use 0 MB)
Set gradient conv threshold to 0.0001
init E= -22.8657603629802
  HOMO = -0.846668587788509  LUMO = 1.20583461938374
cycle= 1 E= -22.8247505899441  delta_E= 0.041  |g|= 0.0113  |ddm|= 0.283
  HOMO = -0.870885382419265  LUMO = 1.20883405665502
cycle= 2 E= -22.8247816130477  delta_E= -3.1e-05  |g|= 0.00126  |ddm|= 0.00875
  HOMO = -0.870790729595039  LUMO = 1.20891944886283
cycle= 3 E= -22.8247820062165  delta_E= -3.93e-07  |g|= 1.91e-06  |ddm|= 0.0011
  HOMO = -0.870790363976892  LUMO = 1.20891957663899
cycle= 4 E= -22.8247820062173  delta_E= -8.03e-13  |g|= 5.

In [4]:
# Find AO's corresponding to atoms
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    #print(ao_idx)
    if ao[0] in (0,1,2,3):
        frag1.append(ao_idx)
        full.append(ao_idx)
    if ao[0] in (4,5,6,7):
        frag2.append(ao_idx)
        full.append(ao_idx)  


frags = [frag1, frag2]
print(frags)
print(len(frag1))
print(len(frag2))

[[0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15]]
8
8


In [5]:
# Define projectors
nbas = Cdocc.shape[0]
X = scipy.linalg.sqrtm(S)
X = np.eye(nbas) 
Pfull = X[:,full]  # non-orthogonal
Pf = []
for f in frags:
    Pf.append(X[:,f])
display(Pfull.shape)

(16, 16)

In [6]:
(Oact, Sact, Vact), (Cenv, Cerr, _) = svd_subspace_partitioning((Cdocc, Csing, Cvirt), Pfull, S)
assert(Cerr.shape[1] == 0)
Cact = np.hstack((Oact, Sact, Vact))
display(Cact)
pyscf.tools.molden.from_mo(mf.mol, "/Users/nicole/My Drive/code/FermiCG-data/he_clusters/Cact.molden", Cact)
print(" Should be 1: ", np.linalg.det(Cact.T @ S @ Cact))

 Partition   16 orbitals into a total of   16 orbitals
            Index   Sing. Val. Space       
                0   1.00000000            2*
                1   1.00000000            2*
                2   1.00000000            2*
                3   1.00000000            2*
                4   1.00000000            0*
                5   1.00000000            0*
                6   1.00000000            2*
                7   1.00000000            2*
                8   1.00000000            2*
                9   1.00000000            2*
               10   1.00000000            0*
               11   1.00000000            0*
               12   1.00000000            0*
               13   1.00000000            0*
               14   1.00000000            0*
               15   1.00000000            0*
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              0              8
              1              0          

array([[ 0.06859315, -0.0694474 , -0.20146951, -0.4668493 , -0.19682619,
        -0.08674694,  0.00988318, -0.193572  ,  0.07880515, -0.48582606,
         0.0270617 , -0.16278789,  0.24319606, -0.08044722, -0.39410592,
         0.92421747],
       [ 0.06118432, -0.05811124, -0.17297097, -0.40545617, -0.17086655,
        -0.07313463,  0.00684766, -0.1702108 , -0.06254005,  0.48994362,
        -0.05053139,  0.17910125, -0.18541648,  0.04889591,  0.35573558,
        -1.02753813],
       [-0.16988826,  0.13367604, -0.20146951,  0.12034673, -0.13253578,
        -0.26622879,  0.37218043,  0.152096  , -0.28462266, -0.34116962,
        -0.1542364 ,  0.72399081,  0.49760085, -0.19126527, -0.39410592,
        -0.39964498],
       [-0.14986039,  0.11471753, -0.17297097,  0.10583966, -0.11383103,
        -0.23056584,  0.32239284,  0.13491577,  0.32168675,  0.28916422,
         0.16279354, -0.75754117, -0.57013295,  0.18446976,  0.35573558,
         0.47396417],
       [ 0.02131128, -0.47923134, -0

 Should be 1:  1.000000000000004


In [7]:
# Project active orbitals onto fragments
init_fspace = []
clusters = []
Cfrags = []
orb_index = 1


for fi,f in enumerate(frags):
    print()
    print(" Fragment: ", f)
    (Of, Sf, Vf), (_, _, _) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)
    # (Of, Sf, Vf), (Oact, Sact, Vact) = svd_subspace_partitioning((Oact, Sact, Vact), Pf[fi], S)

    Cfrags.append(np.hstack((Of, Sf, Vf)))
    ndocc_f = Of.shape[1]
    init_fspace.append((ndocc_f+Sf.shape[1], ndocc_f))
    nmof = Of.shape[1] + Sf.shape[1] + Vf.shape[1]
    clusters.append(list(range(orb_index, orb_index+nmof)))
    orb_index += nmof


# Orthogonalize Fragment orbitals
Cfrags = sym_ortho(Cfrags, S)

# Pseudo canonicalize fragments
Cfrags = canonicalize(Cfrags, F)

Cact = np.hstack(Cfrags)

print("nick: ", np.linalg.svd(Cact.T @ S @ Cact)[1])
# Write Molden files for visualization
#pyscf.tools.molden.from_mo(mf.mol, "Pfull_no.molden", Pfull)
#pyscf.tools.molden.from_mo(mf.mol, "Cact_no.molden", Cact)
#pyscf.tools.molden.from_mo(mf.mol, "Cenv_no.molden", Cenv)
pyscf.tools.molden.from_mo(mf.mol, "/Users/nicole/My Drive/code/FermiCG-data/he_clusters/Pfull.molden", Pfull)
pyscf.tools.molden.from_mo(mf.mol, "/Users/nicole/My Drive/code/FermiCG-data/he_clusters/Cact.molden", Cact)
pyscf.tools.molden.from_mo(mf.mol, "/Users/nicole/My Drive/code/FermiCG-data/he_clusters/Cenv.molden", Cenv)
for i in range(len(frags)):
    pyscf.tools.molden.from_mo(mf.mol, "/Users/nicole/My Drive/code/FermiCG-data/he_clusters/Cfrag_sym_pc%i.molden"%i, Cfrags[i])
#    #pyscf.tools.molden.from_mo(mf.mol, "Cfrag%i.molden"%i, Cfrags[i])


print(" init_fspace = ", init_fspace)
print(" clusters    = ", clusters)


 Fragment:  [0, 1, 2, 3, 4, 5, 6, 7]
 Partition   16 orbitals into a total of    8 orbitals
            Index   Sing. Val. Space       
                0   1.00000000            0*
                1   1.00000000            0*
                2   1.00000000            0*
                3   1.00000000            0*
                4   1.00000000            2*
                5   1.00000000            2*
                6   1.00000000            2*
                7   1.00000000            2*
                8   0.00000248            0
                9   0.00000229            0
               10   0.00000229            0
               11   0.00000212            0
               12   0.00000145            2
               13   0.00000127            2
               14   0.00000127            2
               15   0.00000111            2
  SVD active space has the following dimensions:
  Orbital Block    Environment         Active
              0              4              4
          

In [10]:
print(Cenv.shape)
print(Cact.shape)
d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)

print(h.shape)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)
# The use of d1_embed only really makes sense if it has zero electrons in the
# active space. Let's warn the user if that's not true

S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;

np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/ints_h0", h0)
np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/ints_h1", h1)
np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/ints_h2", h2)
np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/mo_coeffs", Cact)
np.save("/Users/nicole/My Drive/code/FermiCG-data/he_clusters/overlap_mat", S)

Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]
#np.save("Pa", Cact.T @ S @ Pa @ S @ Cact)
#np.save("Pb", Cact.T @ S @ Pb @ S @ Cact)

(16, 0)
(16, 16)
(16, 16)
